# MWS Secondary Target Selection

## MWS_SPECIAL_DDOGIANTS_VERYBRIGHT_SV

Maintainers:
  - David Nidever (selection/catalog)
  - Ting Li
  - Andrew Cooper (secondary target code)

In [1]:
!pwd
!ls ../raw

/global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_DDOGIANTS_VERYBRIGHT_SV/1/code
allgiants_xm_gaia_4DESI_v0.fits


In [2]:
# Parameters

target_class   = 'MWS_SPECIAL_DDOGIANTS_VERYBRIGHT_SV'
input_filename = 'allgiants_xm_gaia_4DESI_v0.fits'

OVERRIDE       = True
REF_CAT        = 'G'
REF_EPOCH      = 2015.5

SELECT_MAG     = 'phot_g_mean_mag'
BRIGHT_LIMIT   =  9.0
FAINT_LIMIT    = 16.0

colmap = {
    'ra':        'RA',
    'dec':       'DEC',
    'pmra':      'PMRA',
    'pmdec':     'PMDEC',
    'source_id': 'REF_ID'
}

{ DESCRIPTION NEEDED }

The selection criteria are:

???

In [3]:
from   importlib import reload
import sys
PYPATH = '/project/projectdirs/desi/external/mws/py/'
if PYPATH not in sys.path: sys.path.append(PYPATH)
import mwsecondary.mwsecondary as mws
reload(mws)

import os
import sys
import numpy as np

targets = mws.SecondaryTargets(target_class,input_filename,ref_cat=REF_CAT,ref_epoch=REF_EPOCH)
targets.print_info()
mask    = targets.select_finite_mag_range(SELECT_MAG,faint_limit=FAINT_LIMIT,bright_limit=BRIGHT_LIMIT)
mask   &= targets.select_finite_range('dec',lower_limit=-30.0,verbose=True)
mask   &= targets.select_unique('source_id',verbose=True)

# Adjust to secondary target data model and write output
header = dict([(_,globals()[_]) for _ in ['BRIGHT_LIMIT','FAINT_LIMIT']])
output = targets.get_output(mask,cols=colmap,write=True,header=header,override=OVERRIDE)
print(output[0:3])

Reading input: /global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_DDOGIANTS_VERYBRIGHT_SV/1/raw/allgiants_xm_gaia_4DESI_v0.fits
Read 291411 rows
First 3 rows:
       ra              dec        ... phot_bp_mean_mag phot_rp_mean_mag
---------------- ---------------- ... ---------------- ----------------
328.519773312368 2.40651951768389 ...        16.265697        15.058078
328.530788027781 2.53707028993252 ...        19.563778        18.097618
328.592711695555 2.42055391398862 ...        15.198254        14.016314

Column names:
['ra', 'dec', 'source_id', 'pmra', 'pmdec', 'phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag']


Mag range: 10.67 < phot_g_mean_mag < 20.88
Mags not finite: 0
Selecting 75258 targets from 291411 (25.83%)
Selecting 291411 from 291411 (100.00%) on dec
286444 of 291411 targets have unique source_id
Removing 4967 duplicates

Writing 73233 rows of output to /global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_DDOGIANTS_VERYBRIGHT_SV/1/inda